In [1]:
# %gui qt
import numpy as np
import mne
import pickle
import sys
import os
# import matplotlib
from multiprocessing import Pool
from tqdm import tqdm
import matplotlib.pyplot as plt

# import vispy
# print(vispy.sys_info())
# BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
# sys.path.append(BASE_DIR)
%matplotlib inline
mne.utils.set_config('MNE_USE_CUDA', 'true')  
mne.cuda.init_cuda(verbose=True)

Now using CUDA device 0
Enabling CUDA with 23.28 GB available memory


In [2]:
baseFolder='./pickled-avg'
files=[f for f in os.listdir(baseFolder) if not f.startswith('.')]

In [3]:
data=pickle.load(open('pickled-avg/OpenBCISession_2020-02-14_11-09-00-SEVEN', 'rb'))

In [4]:
data[0]

[20,
 17560.07,
 17216.7,
 5152.75,
 2391.03,
 4140.66,
 4405.57,
 3411.7,
 228.32,
 6237.81,
 1431.67,
 13483.02,
 -9154.36,
 5237.44,
 8030.2,
 8008.45,
 1521.39,
 510,
 datetime.datetime(1900, 1, 1, 11, 10, 20, 642000),
 {'bpm': -1,
  'ibi': -1,
  'sdnn': -1,
  'sdsd': -1,
  'rmssd': -1,
  'pnn20': -1,
  'pnn50': -1,
  'hr_mad': -1,
  'sd1': -1,
  'sd2': -1,
  's': -1,
  'sd1/sd2': -1,
  'breathingrate': -1,
  'lf': -1,
  'hf': -1,
  'lf/hf': -1,
  'segment_indices': (-1, -1)},
 'sync']

In [5]:
#Naming system for blocks into integers
bloc={
    "sync":1,
    "baseline":2,
    "stressor":3,
    "survey":4,
    "rest":5,
    "slowBreath":6,
    "paced":7
}

def createMNEObj(data, name='Empty'):
    #Create Metadata
    
    sampling_rate = 125
    channel_names = ['Fp1', 'Fp2', 'C3', 'C4', 'P7', 'P8', 'O1', 'O2', 'F7', 'F8', 'F3', 'F4', 'T7', 'T8', 'P3', 'P4',
                    'time', 'bpm', 'ibi', 'sdnn', 'sdsd', 'rmssd', 'pnn20', 'pnn50', 'hr_mad', 'sd1', 'sd2', 's', 'sd1/sd2', 'breathingrate', 'segment_indices1', 'segment_indices2', 'block']
    channel_types = ['eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 
                     'misc', 'misc', 'misc', 'misc', 'misc', 'misc', 'misc', 'misc', 'misc', 'misc', 'misc', 'misc', 'misc', 'misc', 'misc', 'misc', 'stim']
    n_channels = len(channel_types)
    info = mne.create_info(ch_names=channel_names, sfreq=sampling_rate, ch_types=channel_types)
    info['description'] = name
    print(info)
    transformed = []
    start=-1.0
    
    for i in range(len(data)):
        add=[]
        add=data[i][1:17]
#         print(data[i][19].keys())
        if start==-1:
            start=data[i][18].hour*3600 + data[i][18].minute*60 + data[i][18].second + data[i][18].microsecond/1000
            add.append(0.0)
        else:
            tim=data[i][18].hour*3600 + data[i][18].minute*60 + data[i][18].second + data[i][18].microsecond/1000
            add.append(tim-start)
#         add.append(str(data[i][18].hour)+':'+str(data[i][18].minute)+':'+str(data[i][18].second)+':'+str(int(data[i][18].microsecond/1000)))
#         try:
        add.append(data[i][19]['bpm'])
#         except Exception as e: 
#             print(e, i)
#             print(data[i][19])
#             print(len(data))
        
        add.append(data[i][19]['ibi'])
        add.append(data[i][19]['sdnn'])
        add.append(data[i][19]['sdsd'])
        add.append(data[i][19]['rmssd'])
        add.append(data[i][19]['pnn20'])
        add.append(data[i][19]['pnn50'])
        add.append(data[i][19]['hr_mad'])
        add.append(data[i][19]['sd1'])
        add.append(data[i][19]['sd2'])
        add.append(data[i][19]['s'])
        add.append(data[i][19]['sd1/sd2'])
        add.append(data[i][19]['breathingrate'])
        add.append(data[i][19]['segment_indices'][0])
        add.append(data[i][19]['segment_indices'][1])
        add.append(bloc[data[i][20]])
        transformed.append(np.array(add))
    
    transformed=np.array(transformed)
    print(transformed[0])
    #have to convert rows to columns to fit MNE structure
    transformed=transformed.transpose()
    print(transformed[0], transformed[1], transformed[2], transformed[3])
    print(len(transformed[0]))
    loaded=mne.io.RawArray(transformed, info)
    return loaded

In [6]:
raw=createMNEObj(data)

<ipython-input-5-f7810211a598>:21: RuntimeWarning: 2 channel names are too long, have been truncated to 15 characters:
['segment_indices1', 'segment_indices2']
  info = mne.create_info(ch_names=channel_names, sfreq=sampling_rate, ch_types=channel_types)
<ipython-input-5-f7810211a598>:21: RuntimeWarning: Channel names are not unique, found duplicates for: {'segment_indices'}. Applying running numbers for duplicates.
  info = mne.create_info(ch_names=channel_names, sfreq=sampling_rate, ch_types=channel_types)


<Info | 17 non-empty fields
    bads : list | 0 items
    ch_names : list | Fp1, Fp2, C3, C4, P7, P8, O1, O2, F7, ...
    chs : list | 33 items (EEG: 16, MISC: 16, STIM: 1)
    comps : list | 0 items
    custom_ref_applied : bool | False
    description : str | 5 items
    dev_head_t : Transform | 3 items
    events : list | 0 items
    highpass : float | 0.0 Hz
    hpi_meas : list | 0 items
    hpi_results : list | 0 items
    lowpass : float | 62.5 Hz
    meas_date : NoneType | unspecified
    nchan : int | 33
    proc_history : list | 0 items
    projs : list | 0 items
    sfreq : float | 125.0 Hz
    acq_pars : NoneType
    acq_stim : NoneType
    ctf_head_t : NoneType
    dev_ctf_t : NoneType
    device_info : NoneType
    dig : NoneType
    experimenter : NoneType
    file_id : NoneType
    gantry_angle : NoneType
    helium_info : NoneType
    hpi_subsystem : NoneType
    kit_system_id : NoneType
    line_freq : NoneType
    meas_id : NoneType
    proj_id : NoneType
    proj_nam

In [7]:
raw[1][]

SyntaxError: invalid syntax (<ipython-input-7-bb0c725dcd07>, line 1)

In [ ]:
data

In [ ]:
np.transpose(np.transpose(data))

In [8]:
def filt(ind):
    name=files[ind]
    
    data=pickle.load(open('pickled-avg/'+name, 'rb'))
#     if ind==1:
#         pbar = tqdm(total=len(data), position=ind)
    raw=createMNEObj(data)
    print('Created object')
    montage = mne.channels.make_standard_montage('easycap-M1')
    raw.set_montage(montage, raise_if_subset=False)
    mne.io.Raw.filter(raw,l_freq=0.5,h_freq=None)
    print('Done filtering')
    tem=np.transpose(data)
#     for i in tqdm(range(len(data))):
#         if ind==1:
#             pbar.update(1)
#         data[i][k+1]=raw[k][0][0][i]
    for k in range(0, 16):
        tem[k+1]=raw[k][0][0]
    data=np.transpose(tem)
    pickle.dump(data, open('pickled-high/'+name, "wb" ) )

In [ ]:
filt(1)

In [9]:

p = Pool(18)
master=p.map(filt, range(len(files)))

<ipython-input-5-f7810211a598>:21: RuntimeWarning: 2 channel names are too long, have been truncated to 15 characters:
['segment_indices1', 'segment_indices2']
  info = mne.create_info(ch_names=channel_names, sfreq=sampling_rate, ch_types=channel_types)
<ipython-input-5-f7810211a598>:21: RuntimeWarning: Channel names are not unique, found duplicates for: {'segment_indices'}. Applying running numbers for duplicates.
  info = mne.create_info(ch_names=channel_names, sfreq=sampling_rate, ch_types=channel_types)


<Info | 17 non-empty fields
    bads : list | 0 items
    ch_names : list | Fp1, Fp2, C3, C4, P7, P8, O1, O2, F7, ...
    chs : list | 33 items (EEG: 16, MISC: 16, STIM: 1)
    comps : list | 0 items
    custom_ref_applied : bool | False
    description : str | 5 items
    dev_head_t : Transform | 3 items
    events : list | 0 items
    highpass : float | 0.0 Hz
    hpi_meas : list | 0 items
    hpi_results : list | 0 items
    lowpass : float | 62.5 Hz
    meas_date : NoneType | unspecified
    nchan : int | 33
    proc_history : list | 0 items
    projs : list | 0 items
    sfreq : float | 125.0 Hz
    acq_pars : NoneType
    acq_stim : NoneType
    ctf_head_t : NoneType
    dev_ctf_t : NoneType
    device_info : NoneType
    dig : NoneType
    experimenter : NoneType
    file_id : NoneType
    gantry_angle : NoneType
    helium_info : NoneType
    hpi_subsystem : NoneType
    kit_system_id : NoneType
    line_freq : NoneType
    meas_id : NoneType
    proj_id : NoneType
    proj_nam

<ipython-input-5-f7810211a598>:21: RuntimeWarning: 2 channel names are too long, have been truncated to 15 characters:
['segment_indices1', 'segment_indices2']
  info = mne.create_info(ch_names=channel_names, sfreq=sampling_rate, ch_types=channel_types)
<ipython-input-5-f7810211a598>:21: RuntimeWarning: 2 channel names are too long, have been truncated to 15 characters:
['segment_indices1', 'segment_indices2']
  info = mne.create_info(ch_names=channel_names, sfreq=sampling_rate, ch_types=channel_types)
<ipython-input-5-f7810211a598>:21: RuntimeWarning: Channel names are not unique, found duplicates for: {'segment_indices'}. Applying running numbers for duplicates.
  info = mne.create_info(ch_names=channel_names, sfreq=sampling_rate, ch_types=channel_types)
<ipython-input-5-f7810211a598>:21: RuntimeWarning: Channel names are not unique, found duplicates for: {'segment_indices'}. Applying running numbers for duplicates.
  info = mne.create_info(ch_names=channel_names, sfreq=sampling_rate

<Info | 17 non-empty fields
    bads : list | 0 items
    ch_names : list | Fp1, Fp2, C3, C4, P7, P8, O1, O2, F7, ...
    chs : list | 33 items (EEG: 16, MISC: 16, STIM: 1)
    comps : list | 0 items
    custom_ref_applied : bool | False
    description : str | 5 items
    dev_head_t : Transform | 3 items
    events : list | 0 items
    highpass : float | 0.0 Hz
    hpi_meas : list | 0 items
    hpi_results : list | 0 items
    lowpass : float | 62.5 Hz
    meas_date : NoneType | unspecified
    nchan : int | 33
    proc_history : list | 0 items
    projs : list | 0 items
    sfreq : float | 125.0 Hz
    acq_pars : NoneType
    acq_stim : NoneType
    ctf_head_t : NoneType
    dev_ctf_t : NoneType
    device_info : NoneType
    dig : NoneType
    experimenter : NoneType
    file_id : NoneType
    gantry_angle : NoneType
    helium_info : NoneType
    hpi_subsystem : NoneType
    kit_system_id : NoneType
    line_freq : NoneType
    meas_id : NoneType
    proj_id : NoneType
    proj_nam

<ipython-input-5-f7810211a598>:21: RuntimeWarning: 2 channel names are too long, have been truncated to 15 characters:
['segment_indices1', 'segment_indices2']
  info = mne.create_info(ch_names=channel_names, sfreq=sampling_rate, ch_types=channel_types)
<ipython-input-5-f7810211a598>:21: RuntimeWarning: Channel names are not unique, found duplicates for: {'segment_indices'}. Applying running numbers for duplicates.
  info = mne.create_info(ch_names=channel_names, sfreq=sampling_rate, ch_types=channel_types)
<ipython-input-5-f7810211a598>:21: RuntimeWarning: 2 channel names are too long, have been truncated to 15 characters:
['segment_indices1', 'segment_indices2']
  info = mne.create_info(ch_names=channel_names, sfreq=sampling_rate, ch_types=channel_types)


...

<Info | 17 non-empty fields
    bads : list | 0 items
    ch_names : list | Fp1, Fp2, C3, C4, P7, P8, O1, O2, F7, ...
    chs : list | 33 items (EEG: 16, MISC: 16, STIM: 1)
    comps : list | 0 items
    custom_ref_applied : bool | False
    description : str | 5 items
    dev_head_t : Transform | 3 items
    events : list | 0 items
    highpass : float | 0.0 Hz
    hpi_meas : list | 0 items
    hpi_results : list | 0 items
    lowpass : float | 62.5 Hz
    meas_date : NoneType | unspecified
    nchan : int | 33
    proc_history : list | 0 items
    projs : list | 0 items
    sfreq : float | 125.0 Hz
    acq_pars : NoneType
    acq_stim : NoneType
    ctf_head_t : NoneType
    dev_ctf_t : NoneType
    device_info : NoneType
    dig : NoneType
    experimenter : NoneType
    file_id : NoneType
    gantry_angle : NoneType
    helium_info : NoneType
    hpi_subsystem : NoneType
    kit_system_id : NoneType
    line_freq : NoneType
    meas_id : NoneType
    proj_id : NoneType
    proj_nam

<ipython-input-5-f7810211a598>:21: RuntimeWarning: 2 channel names are too long, have been truncated to 15 characters:
['segment_indices1', 'segment_indices2']
  info = mne.create_info(ch_names=channel_names, sfreq=sampling_rate, ch_types=channel_types)
/home/sean/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:60: UserWarning: Warning: converting a masked element to nan.
<ipython-input-5-f7810211a598>:21: RuntimeWarning: Channel names are not unique, found duplicates for: {'segment_indices'}. Applying running numbers for duplicates.
  info = mne.create_info(ch_names=channel_names, sfreq=sampling_rate, ch_types=channel_types)


<Info | 17 non-empty fields
    bads : list | 0 items
    ch_names : list | Fp1, Fp2, C3, C4, P7, P8, O1, O2, F7, ...
    chs : list | 33 items (EEG: 16, MISC: 16, STIM: 1)
    comps : list | 0 items
    custom_ref_applied : bool | False
    description : str | 5 items
    dev_head_t : Transform | 3 items
    events : list | 0 items
    highpass : float | 0.0 Hz
    hpi_meas : list | 0 items
    hpi_results : list | 0 items
    lowpass : float | 62.5 Hz
    meas_date : NoneType | unspecified
    nchan : int | 33
    proc_history : list | 0 items
    projs : list | 0 items
    sfreq : float | 125.0 Hz
    acq_pars : NoneType
    acq_stim : NoneType
    ctf_head_t : NoneType
    dev_ctf_t : NoneType
    device_info : NoneType
    dig : NoneType
    experimenter : NoneType
    file_id : NoneType
    gantry_angle : NoneType
    helium_info : NoneType
    hpi_subsystem : NoneType
    kit_system_id : NoneType
    line_freq : NoneType
    meas_id : NoneType
    proj_id : NoneType
    proj_nam

/home/sean/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:60: UserWarning: Warning: converting a masked element to nan.


[ 1.756007e+04  1.721670e+04  5.152750e+03  2.391030e+03  4.140660e+03
  4.405570e+03  3.411700e+03  2.283200e+02  6.237810e+03  1.431670e+03
  1.348302e+04 -9.154360e+03  5.237440e+03  8.030200e+03  8.008450e+03
  1.521390e+03  0.000000e+00 -1.000000e+00 -1.000000e+00 -1.000000e+00
 -1.000000e+00 -1.000000e+00 -1.000000e+00 -1.000000e+00 -1.000000e+00
 -1.000000e+00 -1.000000e+00 -1.000000e+00 -1.000000e+00 -1.000000e+00
 -1.000000e+00 -1.000000e+00  1.000000e+00]
[17560.07 17242.04 17505.35 ... 17545.67 16875.68 17536.64] [17216.7  17052.53 17212.77 ... 17820.67 17206.71 17815.68] [5152.75 5267.52 5161.53 ... 4904.58 4756.03 4912.67] [2391.03 2337.59 2375.57 ... 2081.77 1719.61 2081.51]
201227
Creating RawArray with float64 data, n_channels=33, n_times=201227
    Range : 0 ... 201226 =      0.000 ...  1609.808 secs
Ready.
Created object
DigMontage is a superset of info. 58 in DigMontage will be ignored. The ignored channels are: {'C1', 'CP6', 'TP8', 'FC5', 'C5', 'CP4', 'AF3', 'FC4', 

In [20]:
data=pickle.load(open('pickled-filt/OpenBCISession_2020-02-14_11-09-00-SEVEN', 'rb'))

In [23]:
data[0]

array([20, -3.801403636316536e-13, -3.26316751397826e-12,
       -1.2523315717771766e-12, -4.596323321948148e-13,
       -3.161915174132446e-13, -1.4761525335416081e-12,
       -7.602807272633072e-13, -1.438849039914203e-13,
       -1.7479351299698465e-12, -1.0658141036401503e-14,
       -7.922551503725117e-13, -2.0383694732117874e-12,
       -2.1280754936015e-12, -1.5825118993006981e-12,
       -1.3447021274259896e-12, -2.9665159217984183e-13, 510,
       datetime.datetime(1900, 1, 1, 11, 10, 20, 642000),
       {'bpm': -1, 'ibi': -1, 'sdnn': -1, 'sdsd': -1, 'rmssd': -1, 'pnn20': -1, 'pnn50': -1, 'hr_mad': -1, 'sd1': -1, 'sd2': -1, 's': -1, 'sd1/sd2': -1, 'breathingrate': -1, 'lf': -1, 'hf': -1, 'lf/hf': -1, 'segment_indices': (-1, -1)},
       'sync'], dtype=object)